In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
from Functions import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.4f}'.format

import pylab as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

from sklearn.preprocessing import StandardScaler, MinMaxScaler  # normalizacion
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

#%matplotlib inline

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import LogisticRegression as LogReg

from sklearn.svm import SVR

#!pip install xgboost
#!pip install catboost
#!pip install lightgbm
#!pip install h2o

from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

import h2o
from h2o.automl import H2OAutoML

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier

from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge

In [10]:
def ml1():
    
    models = {'RFR':{'MODEL':RFR(),'PARAM':{'n_estimators': [50,150,500],'max_depth':[1,10,15],'min_weight_fraction_leaf':[0.0,0.1,0.2]}}}
              #'XGB':{'MODEL':XGBR(),'PARAM':{'n_estimators': [10, 50, 100, 150, 200, 500],'max_depth': [1, 5,6, 10, 15, 20],'learning_rate':[0.001,0.002,0.01,0.05] }},
              #'SVR':{'MODEL':SVR(),'PARAM':{'kernel':['rbf','poly','linear']}},
              #'CTR':{'MODEL':CTR(),'PARAM:':{'depth' : [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations': [30, 50, 100],'subsample' : [0.5, 0.7, 1.0]}},
              #'GaussianNB':{'MODEL':GaussianNB(),'PARAM':{'var_smoothing':[1e-09]}},
   
              #'Lasso':{'MODEL':Lasso(),'PARAM':{'alpha':[0.3,0.5,0.7,0.9,1.0],'max_iter':[800,1000,1200]}}}
    
    rmse = []
    name = []
    #b = []
    score = []
    dfmodels = pd.DataFrame()
    
    for m in models:
        x = models[m]["MODEL"]
        p = models[m]["PARAM"]
        y_pred=grid(x,p).predict(X_test)
        #best= grid(x,p).best_params_   
        #sco = grid(x,p).best_score_
        MSE = mse(y_test, y_pred, squared=False)
        #score.append(sco)
        #b.append(best)
        rmse.append(MSE)
        name.append(m)
        importance = grid(x,p).feature_importances_
        for i,v in enumerate(importance):
            print('Feature: %0d, Score: %.5f' % (i,v))
    dfmodels['Modelo'] = name
    dfmodels['RMSE'] = rmse
    #dfmodels['Best_Parametres'] = b
    #dfmodels['bestsco'] = score                        
    dfmodels.sort_values("RMSE",ascending=True,inplace=True,ignore_index=True)
    print(f'model {dfmodels.Modelo[0]} rmse {dfmodels.RMSE[0]} ')
    
    
    return dfmodels


def grid(modelo, param):
    
    g=GridSearchCV(modelo, # modelo de sklearn
                   param,  # dictio de parametros
                   cv=5,   # nº de cortes del cross-validation
                   return_train_score=True, # error en entrenamiento para checkear
                   n_jobs=-1  # usa todos los nucleos disponibles
                  )

    g.fit(X_train, y_train)
    print('Acierto test: {:.2f}'.format(g.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(g.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(g.best_params_))
    print('Modelo: {}'.format(modelo))
    print('Mejor acierto cv: {:.2f}'.format(g.best_score_))

    return g.best_estimator_.fit(X_train, y_train)

In [4]:
def machine_learning():
    
    modelos = {'LinReg':LinReg(), 'Lasso':Lasso(), 'Ridge':Ridge(), 'Elastic':ElasticNet(),'RFR':RFR()}
    
    nombres = ['LinReg', 'Lasso', 'Ridge', 'Elastic','RFR']
    
    RMSE = []
    R2_test = []
    R2_train = []

    resumen_modelos = pd.DataFrame()

    for m in modelos:
     
            
        mod = modelos[m]
        mod.fit(X_train, y_train)

        y_pred=mod.predict(X_test)
        error = mse(y_test, y_pred, squared=False)

        y_pred = mod.predict(X_test)
        R2_ts = mod.score(X_test, y_test)
        y_pred = mod.predict(X_train) 
        R2_tr = mod.score(X_train, y_train)

        RMSE.append(int(error))
        R2_test.append(R2_ts)
        R2_train.append(R2_tr)

    resumen_modelos['Modelo'] = nombres
    resumen_modelos['RMSE'] = RMSE
    resumen_modelos['R2_test'] = R2_test
    resumen_modelos['R2_train'] = R2_train
    
    dict_errores = dict(zip(nombres, RMSE))
    best = list(dict_errores.keys())[list(dict_errores.values()).index(min(RMSE))] 
    
    modelo = modelos[best]
    
    modelo.fit(X_train, y_train)
    
    y_pred = modelo.predict(X_test)
    R2_test = modelo.score(X_test, y_test)
    y_pred = modelo.predict(X_train) 
    R2_train = modelo.score(X_train, y_train)
    
    error = round(min(RMSE),2)
    R2_train = round(R2_train, 4)
    R2_test = round(R2_test, 4)
    
    if R2_train > (1.15*R2_test):
        print(f'Best Model: {best}, RMSE = {error}, R2_train = {R2_train}, R2_test = {R2_test}, OVERFITING (modifica datos)')
    
    elif R2_train > R2_test:
        print(f'Best Model: {best}, RMSE = {error}, R2_train = {R2_train}, R2_test = {R2_test}, LO NORMAL')
    
    elif R2_train < R2_test:
        print(f'Best Model: {best}, RMSE = {error}, R2_train = {R2_train}, R2_test = {R2_test}, UNDERFITING (dame más datos)')
        
    return modelos[resumen_modelos["Modelo"][4]]

### We select several factors to use for our predictive model and normalize the predictive variables for each of the factors

In [7]:
factors = ["Adolescent birth rate (per 1000 women aged 15-19 years)","Total NCD Deaths (in thousands)","Estimates of rates of homicides per 100 000 population","Age-standardized suicide rates (per 100 000 population)","Prevalence of controlled hypertension among adults aged 30-79 years with hypertension, age-standardized","gdp/percap","per_capita-Chronic Respiratory Diseases"]

In [74]:
df1`""

,Unnamed: 0,Malaria,Interpersonal Violence,HIV/AIDS,Drug Use Disorders,Exposure to Forces of Nature,Environmental Heat and Cold Exposure,Conflict and Terrorism,Protein-Energy Malnutrition,Acute Hepatitis,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Alcohol Use Disorders,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Acute Hepatitis,gdp/percap,Out-of-pocket expenditure as percentage of current health expenditure (CHE) (%),Domestic private health expenditure (PVT-D) per capita in US$,Adolescent birth rate (per 1000 women aged 15-19 years),Deaths due to tuberculosis among HIV-negative people (per 100 000 population),Age-standardized suicide rates (per 100 000 population),Total NCD Deaths (in thousands),Estimated number of road traffic deaths,Estimated road traffic death rate (per 100 000 population),"Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)_x","Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)_y",Mortality rate attributed to unintentional poisoning (per 100 000 population),Crude suicide rates (per 100 000 population),Estimates of number of homicides,Estimates of rates of homicides per 100 000 population,Yellow fever - number of reported cases,Mumps - number of reported cases,"Prevalence of controlled hypertension among adults aged 30-79 years with hypertension, age-standardized",num_nan
0,0,2883.0000,3277.0000,111.0000,202.0000,1286.0000,53.0000,977.0000,2746.0000,4251.0000,0.0001,0.0001,0.0001,0.0000,0.0000,0.0000,0.0001,0.0005,0.0000,0.0000,0.0001,0.0001,0.0003,0.0003,0.0001,0.0002,218.0500,85.3800,13.4900,12.3000,57.0000,7.6900,44342.7300,733.7519,14.9154,0.0079,0.0000,0.8312,5.3500,2010.9130,10.1756,0.0000,0.0000,4.9000,19
1,1,2188.0000,3458.0000,118.0000,216.0000,137.0000,55.0000,864.0000,2984.0000,4373.0000,0.0001,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0005,0.0000,0.0000,0.0001,0.0001,0.0003,0.0003,0.0001,0.0002,246.6268,86.0700,14.6600,8.1000,58.0000,7.7800,44521.9100,3556.0000,22.5714,0.0079,0.0000,4.2060,4.9500,2137.9780,10.4018,0.0000,0.0000,5.3000,19
2,2,773.0000,3479.0000,125.0000,229.0000,18.0000,57.0000,1049.0000,2978.0000,4345.0000,0.0000,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0005,0.0000,0.0000,0.0001,0.0001,0.0003,0.0003,0.0001,0.0002,269.4800,84.5300,17.2500,135.7000,51.0000,7.7700,47583.4600,3757.0000,15.1941,0.0079,0.0000,0.8695,5.3900,330.4790,16.2720,0.0000,0.0000,5.6000,18
3,3,545.0000,3506.0000,132.0000,239.0000,561.0000,78.0000,2114.0000,2824.0000,4194.0000,0.0000,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0001,0.0001,0.0001,0.0003,0.0003,0.0001,0.0002,324.8945,78.9700,18.8700,8.2000,47.0000,7.7800,95631.1200,3891.0000,15.1671,0.0008,0.0000,0.7984,4.5400,2118.9150,2.8048,0.0000,0.0000,5.9000,15
4,4,414.0000,3609.0000,140.0000,245.0000,382.0000,57.0000,5677.0000,2675.0000,3870.0000,0.0000,0.0001,0.0001,0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0002,0.0001,0.0001,0.0003,0.0003,0.0001,0.0001,379.2021,76.8200,21.3900,4.3000,43.0000,7.7500,97079.5500,3101.3450,22.7640,0.0079,0.0000,1.9292,4.4400,2500.9800,9.4616,0.0000,0.0000,6.3000,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,3073,2518.0000,1302.0000,29162.0000,104.0000,16.0000,37.0000,13.0000,2990.0000,146.0000,0.0002,0.0001,0.0001,0.0020,0.0000,0.0000,0.0000,0.0004,0.0000,0.0000,0.0002,0.0002,0.0002,0.0002,0.0001,0.0000,1403.3481,25.7900,59.1500,1.6333,7.9000,30.7352,26388.9800,4162.5460,63.3687,3.8400,0.0500,5.1448,23.8332,431.2020,13.4306,0.00

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
dic1 = {}
for i in factors:
    df1=pd.read_csv(f"C:\\Users\\pabli\\Desktop\\Health-Database\\data\\Df_modeling2\\threshold4.csv")
    df1.drop(columns="Unnamed: 0",inplace=True)
    df1_ = df1.drop(columns=i).columns
    scaler=StandardScaler()
    df1[df1_]=scaler.fit_transform(df1[df1_])
    X = df1.drop(columns=i)
    y = df1[i]
    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.9, test_size=0.1, random_state=42)
    rfe = RFE(estimator=machine_learning(), n_features_to_select=5)
    rfe.fit(X, y)
    for t in range(X.shape[1]):
        print('Column: %d, Selected %s, Rank: %.3f' % (t, rfe.support_[t], rfe.ranking_[t]))
    d1 = []
    for t in range(X.shape[1]):
        if rfe.support_[t] == True:
            d1.append(t)
    dic1[y.name]= d1
    df = df1.iloc[:,dic1[y.name]]
    try:
        df.to_csv(f"C:\\Users\\pabli\\Desktop\\Health-Database\\data\\thresh4_models\\{i}.csv")
    except:
        print(i)
    
        
    


    
    

In [5]:
lst = [1,2,3,4,5]

### For each of the 5 factors we can save the necessary columns with the help of a feature selector, then with the help of a function we created in our Functions.py we check which regressor is the optimal one 

In [12]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
for r in lst:
    dic = {}
    for i in factors:
        df1=pd.read_csv(f"C:\\Users\\pabli\\Desktop\\Health-Database\\data\\Df_modeling2\\threshold{r}.csv")
        df1.drop(columns="Unnamed: 0",inplace=True)
        try:   
            df1_ = df1.drop(columns=i).columns
            scaler=StandardScaler()
            df1[df1_]=scaler.fit_transform(df1[df1_])
            X = df1.drop(columns=i)
            y = df1[i]
            X_train, X_test, y_train, y_test = tts(X, y, train_size=0.9, test_size=0.1, random_state=42)
            rfe = RFE(estimator=machine_learning(), n_features_to_select=5)
            rfe.fit(X, y)
            for t in range(X.shape[1]):
                print('Column: %d, Selected %s, Rank: %.3f' % (t, rfe.support_[t], rfe.ranking_[t]))
            d1 = []
            for t in range(X.shape[1]):
                if rfe.support_[t] == True:
                    d1.append(t)
            dic[y.name]= d1
            df = df1.iloc[:,dic[y.name]]
            try:    
                df.to_csv(f"C:\\Users\\pabli\\Desktop\\Health-Database\\data\\thresh{r}_models\\{i}.csv")
            except:
                print(i)
        except:
            print(i)
    
    

Best Model: RFR, RMSE = 10, R2_train = 0.9424, R2_test = 0.5608, OVERFITING (modifica datos)
Column: 0, Selected False, Rank: 4.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected False, Rank: 14.000
Column: 3, Selected False, Rank: 20.000
Column: 4, Selected False, Rank: 2.000
Column: 5, Selected False, Rank: 15.000
Column: 6, Selected True, Rank: 1.000
Column: 7, Selected True, Rank: 1.000
Column: 8, Selected True, Rank: 1.000
Column: 9, Selected False, Rank: 9.000
Column: 10, Selected False, Rank: 21.000
Column: 11, Selected False, Rank: 19.000
Column: 12, Selected False, Rank: 13.000
Column: 13, Selected False, Rank: 12.000
Column: 14, Selected False, Rank: 7.000
Column: 15, Selected False, Rank: 5.000
Column: 16, Selected False, Rank: 3.000
Column: 17, Selected False, Rank: 6.000
Column: 18, Selected False, Rank: 16.000
Column: 19, Selected False, Rank: 10.000
Column: 20, Selected False, Rank: 8.000
Column: 21, Selected False, Rank: 18.000
Column: 22, Selected True, Rank

Best Model: RFR, RMSE = 2911, R2_train = 0.9957, R2_test = 0.9768, LO NORMAL
Column: 0, Selected False, Rank: 27.000
Column: 1, Selected False, Rank: 12.000
Column: 2, Selected False, Rank: 14.000
Column: 3, Selected False, Rank: 26.000
Column: 4, Selected False, Rank: 10.000
Column: 5, Selected False, Rank: 25.000
Column: 6, Selected False, Rank: 15.000
Column: 7, Selected False, Rank: 22.000
Column: 8, Selected False, Rank: 5.000
Column: 9, Selected False, Rank: 9.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected False, Rank: 4.000
Column: 12, Selected False, Rank: 20.000
Column: 13, Selected False, Rank: 24.000
Column: 14, Selected False, Rank: 2.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected True, Rank: 1.000
Column: 17, Selected False, Rank: 13.000
Column: 18, Selected True, Rank: 1.000
Column: 19, Selected False, Rank: 11.000
Column: 20, Selected False, Rank: 7.000
Column: 21, Selected False, Rank: 21.000
Column: 22, Selected False, Rank: 6.000
Colu

Best Model: RFR, RMSE = 2380, R2_train = 0.9961, R2_test = 0.9834, LO NORMAL
Column: 0, Selected False, Rank: 33.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected False, Rank: 19.000
Column: 3, Selected False, Rank: 17.000
Column: 4, Selected False, Rank: 22.000
Column: 5, Selected False, Rank: 14.000
Column: 6, Selected False, Rank: 32.000
Column: 7, Selected False, Rank: 18.000
Column: 8, Selected False, Rank: 25.000
Column: 9, Selected False, Rank: 6.000
Column: 10, Selected False, Rank: 9.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected False, Rank: 11.000
Column: 13, Selected False, Rank: 30.000
Column: 14, Selected False, Rank: 5.000
Column: 15, Selected False, Rank: 31.000
Column: 16, Selected False, Rank: 3.000
Column: 17, Selected True, Rank: 1.000
Column: 18, Selected True, Rank: 1.000
Column: 19, Selected False, Rank: 2.000
Column: 20, Selected False, Rank: 16.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected False, Rank: 12.000
Colum

### 